In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import gzip
import grader

# Time Series Data: Predict Temperature
Time series prediction presents its own challenges which are different from machine-learning problems.  As with many other classes of problems, there are a number of common features in these predictions.

## A note on scoring
It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!

## Fetch the data:

In [3]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'train.txt.gz'

download: s3://dataincubator-course/mldata/train.txt.gz to ./train.txt.gz


The columns of the data correspond to the
  - year
  - month
  - day
  - hour
  - temp
  - dew_temp
  - pressure
  - wind_angle
  - wind_speed
  - sky_code
  - rain_hour
  - rain_6hour
  - city

This function will read the data from a file handle into a Pandas DataFrame.  Feel free to use it, or to write your own version to load it in the format you desire.

In [3]:
def load_stream(stream):
    return pd.read_table(stream, sep=' *',
                         names=['year', 'month', 'day', 'hour', 'temp',
                                'dew_temp', 'pressure', 'wind_angle', 
                                'wind_speed', 'sky_code', 'rain_hour',
                                'rain_6hour', 'city'])

In [68]:
df = load_stream(gzip.open('train.txt.gz', 'r'))

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


## The temperature is reported in tenths of a degree Celcius.  However, not all the values are valid.  Examine the data, and remove the invalid rows.

In [69]:
df.head(50)
df = df[(df['temp']>-500)]
df = df.reset_index(drop=True)
df.head()

,year,month,day,hour,temp,dew_temp,pressure,wind_angle,wind_speed,sky_code,rain_hour,rain_6hour,city
0,2000,1,1,0,-11,-72,10197,220,26,4,0,0,bos
1,2000,1,1,1,-6,-78,10206,230,26,2,0,-9999,bos
2,2000,1,1,2,-17,-78,10211,230,36,0,0,-9999,bos
3,2000,1,1,3,-17,-78,10214,230,36,0,0,-9999,bos
4,2000,1,1,4,-17,-78,10216,230,36,0,0,-9999,bos


In [75]:
df.columns

Index([u'year', u'month', u'day', u'hour', u'temp', u'dew_temp', u'pressure',
       u'wind_angle', u'wind_speed', u'sky_code', u'rain_hour', u'rain_6hour',
       u'city'],
      dtype='object')

In [6]:
cities = df.groupby('city')
mycities = cities.groups.keys()
print mycities

['chi', 'bal', 'bos', 'phi', 'nyc']


In [7]:
df1 = df.head()
df1['new'] = df1.apply(lambda x: x['temp']+1, axis=1)
df1 = df1.drop('new', axis=1)
li = ['month', 'hour']
df1[li]

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,month,hour
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4


In [30]:
rows = df.shape[0]
y_pred = [None,]*rows
y_pred[-3:]
df_train = df[ df['city'] == 'bos' ]
index = df_train.index.values
print index.shape
index[:10]

(105169,)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [19]:
df1 = df.head().copy()
df1.city[2] = 'pov'
df1.city[4] = 'pov'
df2a = df1[df1['city']=='bos']
df2b = df1[df1['city']!='bos']
#print df2a
#print df2b
df2 = pd.concat([df2a,df2b])
df2 = df2.sort_index()
print df2

print df.head()

   year  month  day  hour  temp  dew_temp  pressure  wind_angle  wind_speed  \
0  2000      1    1     0   -11       -72     10197         220          26   
1  2000      1    1     1    -6       -78     10206         230          26   
2  2000      1    1     2   -17       -78     10211         230          36   
3  2000      1    1     3   -17       -78     10214         230          36   
4  2000      1    1     4   -17       -78     10216         230          36   

   sky_code  rain_hour  rain_6hour city  
0         4          0           0  bos  
1         2          0       -9999  bos  
2         0          0       -9999  pov  
3         0          0       -9999  bos  
4         0          0       -9999  pov  
   year  month  day  hour  temp  dew_temp  pressure  wind_angle  wind_speed  \
0  2000      1    1     0   -11       -72     10197         220          26   
1  2000      1    1     1    -6       -78     10206         230          26   
2  2000      1    1     2   -17     

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


We will focus on using the temporal elements to predict the temperature.

## Per city model

It makes sense for each city to have it's own model.  Build a "groupby" estimator that takes an estimator factory as an argument and builds the resulting "groupby" estimator on each city.  That is, `fit` should create and fit a model per city, while the `predict` method should look up the corresponding model and perform a predict on each.  An estimator factory is something that returns an estimator each time it is called.  It could be a function or a class.

In [70]:
from sklearn import base

class GroupbyEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, column, estimator_factory):
        # column is the value to group by; estimator_factory can be
        # called to produce estimators
        self.column = column
        self.estimator_factory = estimator_factory
    
    def fit(self, X, y):
        # Create an estimator and fit it with the portion in each group
        cats = X.groupby(self.column)
        mycats = cats.groups.keys()
        self.est = dict()
        for cat in mycats:
            X_train = X[ X[self.column] == cat ]
            y_train = y[ X[self.column] == cat ]
            myest = self.estimator_factory()
            myest.fit(X_train, y_train)
            self.est[cat] = myest
        return self

    def predict(self, X):
        # Call the appropriate predict method for each row of X
        #self.y_pred = X.apply(lambda r: self.est[r[self.column]].predict(r), axis=1)
        cats = X.groupby(self.column)
        mycats = cats.groups.keys()
        rows = X.shape[0]
        y_pred = [None,]*rows
        for cat in mycats:
            if cat in self.est.keys():
                X_train = X[ X[self.column] == cat ]
                X_index = X_train.index.values
                estimator = self.est[cat]
                pred = estimator.predict(X_train)
                for i, p in enumerate(pred):
                    ind = X_index[i]
                    y_pred[ind] = p
        return y_pred

# Questions

For each question, build a model to predict the temperature in a given city at a given time.  You will be given a list of records, each a string in the same format as the lines in the training file.  Return a list of predicted temperatures, one for each incoming record.  (As you can imagine, the temperature values will be stripped out in the actual text records.)

## 1. month_hour_model
Seasonal features are nice because they are relatively safe to extrapolate into the future. There are two ways to handle seasonality.  

The simplest (and perhaps most robust) is to have a set of indicator variables. That is, make the assumption that the temperature at any given time is a function of only the month of the year and the hour of the day, and use that to predict the temperature value.

**Question**: Should month be a continuous or categorical variable?  (Recall that [one-hot encoding](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) is useful to deal with categorical variables.)

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor

In [77]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        self.y = y
        self.X = X
        return self
    
    def transform(self, X):
        data = X[self.col_names]
        return data
         
class EncoderTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return OneHotEncoder().fit_transform(X).toarray()
    
def season_factory():
    # A single estimator or a pipeline
    pipeline = Pipeline([
        ('trans', ColumnSelectTransformer(['month','hour'])),
        ('enc', EncoderTransformer() ), 
        ('reg', Ridge(alpha=5.0))
        #('reg', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls'))
    ])
    return pipeline

season_model = GroupbyEstimator('city', season_factory).fit(df, df['temp'])

In [78]:

pred = season_model.predict(df)


In [79]:
pred

[-9.371587073941285,
 -14.0882070141312,
 -17.888567502249359,
 -21.600449800673516,
 -24.640905016502444,
 -30.321800692449131,
 -30.950575990766509,
 -33.789291876881435,
 -36.072782591050071,
 -38.1232421882398,
 -39.334697901978416,
 -37.343847295366373,
 -31.83307294449736,
 -23.217322720396737,
 -13.926487192374452,
 -5.1482332124141408,
 1.7387137464779556,
 6.8935047397367271,
 10.116249291848476,
 11.391789369023058,
 10.547054676286621,
 7.1784927939004319,
 1.9381827163811636,
 -3.7097821514253724,
 -9.371587073941285,
 -14.0882070141312,
 -17.888567502249359,
 -21.600449800673516,
 -24.640905016502444,
 -30.321800692449131,
 -30.950575990766509,
 -33.789291876881435,
 -36.072782591050071,
 -38.1232421882398,
 -39.334697901978416,
 -37.343847295366373,
 -31.83307294449736,
 -23.217322720396737,
 -13.926487192374452,
 -5.1482332124141408,
 1.7387137464779556,
 6.8935047397367271,
 10.116249291848476,
 11.391789369023058,
 10.547054676286621,
 7.1784927939004319,
 1.9381827163

You will need to write a function that makes predictions from a **list of strings.**  You can either create a pipeline with a transformer and the `season_model`, or you can write a helper function to convert the lines to the format you expect.

In [80]:
def month_hour_model(x):
    col_names = [u'year', u'month', u'day', u'hour', u'temp', u'dew_temp', u'pressure',
       u'wind_angle', u'wind_speed', u'sky_code', u'rain_hour', u'rain_6hour', u'city']
    data = []
    nn = len(x)
    for i in range(nn):
        row = x[i]
        li = row.split()
        for d in range(12):
            li[d] = int(li[d])
        data.append(li)
    df = pd.DataFrame(data, columns=col_names)
    return season_model.predict(df)

grader.score('ts__month_hour_model', month_hour_model)  #lambda x: [0] * len(x)

Your score:  1.00011375612


## 2. fourier_model
Since we know that temperature is roughly sinusoidal, we know that a reasonable model might be

$$ y_t = y_0 \sin\left(2\pi\frac{t - t_0}{T}\right) + \epsilon $$

where $k$ and $t_0$ are parameters to be learned and $T$ is one year for seasonal variation.  While this is linear in $y_0$, it is not linear in $t_0$. However, we know from Fourier analysis, that the above is
equivalent to

$$ y_t = A \sin\left(2\pi\frac{t}{T}\right) + B \cos\left(2\pi\frac{t}{T}\right) + \epsilon $$

which is linear in $A$ and $B$.

Create a model containing sinusoidal terms on one or more time scales, and fit it to the data using a **linear regression.**

In [115]:
from datetime import datetime
import math

class TimeTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, T0, T1):
        self.T0 = T0
        self.T1 = T1
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        rows = X.shape[0]
        data = []
        #print X.iloc[0]
        for i in range(rows):
            row = X.iloc[i]
            mt= datetime(row.year, row.month, row.day, row.hour)
            ytd = mt.timetuple().tm_yday
            t = ytd*24 + row.hour
            x1 = math.sin(2.0 * math.pi * t / self.T0)
            x2 = math.cos(2.0 * math.pi * t / self.T0)
            x3 = math.sin(2.0 * math.pi * t / self.T1)
            x4 = math.cos(2.0 * math.pi * t / self.T1)
            data.append([x1,x2,x3,x4])
        return data

In [117]:
def time_factory():
    # A single estimator or a pipeline
    pipeline = Pipeline([
        ('trans', TimeTransformer(365*24, 24)),
        ('reg', Ridge(alpha=5.0))
    ])
    return pipeline

fourier_model = GroupbyEstimator('city', time_factory).fit(df, df['temp'])

In [118]:
f_pred = fourier_model.predict(df)
print f_pred[:100]

[11.399977836473809, 5.4261244772352342, -0.87193344773318415, -7.0674899634718145, -12.740820576754516, -17.507785805392245, -21.046009647843405, -23.116848973689159, -23.581656657318561, -22.411230149139371, -19.687801579800933, -15.599433775114861, -10.427204083996045, -4.5260494203699295, 1.6994221022151947, 7.8225045461423832, 13.423473455470585, 18.118189385237798, 21.584276371071994, 23.583091319665698, 23.975987142461676, 22.733761326863444, 19.938646040458238, 15.77870414593751, 10.535013029038282, 4.5625096404499601, -1.7341978695731228, -7.9284035267658624, -13.600382838597284, -18.365996323573384, -21.902867980848058, -23.972354680698103, -24.43580929820844, -23.26402928448293, -20.539246770867223, -16.449524583869476, -11.275940073101438, -5.3734301531854101, 0.85339706163294693, 6.9778356330393905, 12.580161104394932, 17.276234030040044, 20.743678444904262, 22.743851254981934, 23.13810537101719, 21.897238279714841, 19.103482147962978, 14.944899837753951, 9.702568734125307

In [101]:
from datetime import datetime
df1 = df.head(20)
#df1['time'] = datetime(df1['year'], df1['month'], df1['day'], df1['hour'])
row = df1.loc[0]
row.year
print row
mt= datetime(row.year, row.month, row.day, row.hour)
yeartodate = mt.timetuple().tm_yday
print yeartodate

year           2000
month             1
day               1
hour              0
temp            -11
dew_temp        -72
pressure      10197
wind_angle      220
wind_speed       26
sky_code          4
rain_hour         0
rain_6hour        0
city            bos
Name: 0, dtype: object
1


In [89]:
mt= datetime(row.year, row.month, row.day, row.hour)
ytd = mt.timetuple().tm_yday
t = ytd*24 + row.hour
print t

25


In [128]:
df1 = df.head(10).copy()
#df1['time'] = pd.to_datetime(df1[['year','month','day','hour']])
df1['date'] = pd.to_datetime(2000*10000+df1.month*100+df1.day, format='%Y%m%d')
df1['days'] = (df1.date - np.datetime64('2000-01-01')) / np.timedelta64(1, 'D')
df1['total_hour'] = np.ceil(df1.days*24 + df1.hour)
df1['x1'] = np.sin(2.0 * math.pi * df1.total_hour / 365*24)
df1['x2'] = np.cos(2.0 * math.pi * df1.total_hour / 365*24)
df1['x3'] = np.sin(2.0 * math.pi * df1.total_hour / 24)
df1['x4'] = np.cos(2.0 * math.pi * df1.total_hour / 24)

In [136]:
df.head()

,year,month,day,hour,temp,dew_temp,pressure,wind_angle,wind_speed,sky_code,rain_hour,rain_6hour,city
0,2000,1,1,0,-11,-72,10197,220,26,4,0,0,bos
1,2000,1,1,1,-6,-78,10206,230,26,2,0,-9999,bos
2,2000,1,1,2,-17,-78,10211,230,36,0,0,-9999,bos
3,2000,1,1,3,-17,-78,10214,230,36,0,0,-9999,bos
4,2000,1,1,4,-17,-78,10216,230,36,0,0,-9999,bos


In [145]:
from datetime import datetime
import math

class TimeTransformer_pd(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, T0, T1):
        self.T0 = T0
        self.T1 = T1
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        date = pd.to_datetime(2000*10000+X.month*100+X.day, format='%Y%m%d')
        days = (date - np.datetime64('2000-01-01')) / np.timedelta64(1, 'D')
        total_hour = np.ceil(days*24 + X.hour)
        x1 = np.sin(2.0 * math.pi * total_hour / self.T0)
        x2 = np.cos(2.0 * math.pi * total_hour / self.T0)
        x3 = np.sin(2.0 * math.pi * total_hour / self.T1)
        x4 = np.cos(2.0 * math.pi * total_hour / self.T1)
        data = pd.concat([x1,x2,x3,x4],axis=1)
        data = data.rename(columns={0:'x1', 1:'x2', 2:'x3', 3:'x4'})
        #print data.head()
        return data
    
def time_factory_pd():
    # A single estimator or a pipeline
    pipeline = Pipeline([
        ('trans', TimeTransformer_pd(365*24, 24)),
        ('reg', Ridge(alpha=5.0))
    ])
    return pipeline

fourier_model_pd = GroupbyEstimator('city', time_factory_pd).fit(df, df['temp'])

In [146]:
y_pred_pd = fourier_model_pd.predict(df)
y_pred_pd[:100]

[12.102704826214236,
 6.1275096978844061,
 -0.17185981988531296,
 -6.3686737714439232,
 -12.043187398660862,
 -16.811246050081024,
 -20.350464683756186,
 -22.422197874665088,
 -22.887803104775642,
 -21.718089019565852,
 -18.995304769863552,
 -14.907534866447989,
 -9.7358815300151775,
 -3.8353080392733716,
 2.389551432932393,
 8.5119669703249201,
 14.112193852007721,
 18.806077464855491,
 22.271232805630135,
 24.269014487112401,
 24.660780029319113,
 23.41733811570775,
 20.620937935039848,
 16.459664035959264,
 11.214618676981075,
 5.240765174563748,
 -1.0572622607429736,
 -7.2527336750258229,
 -12.925904310090743,
 -17.692619514644733,
 -21.230494248155097,
 -23.300883086467408,
 -23.765143511776117,
 -22.594084170519963,
 -19.869954213886203,
 -15.780838153682481,
 -10.607838211500706,
 -4.7059176658027866,
 1.5202893089311118,
 7.6440527957164761,
 13.245628073952844,
 17.940860528832815,
 21.407365156595958,
 23.406496569802954,
 23.799612287305337,
 22.557520992140738,
 19.76247187

In [134]:
f_pred[:100]

[11.399977836473809,
 5.4261244772352342,
 -0.87193344773318415,
 -7.0674899634718145,
 -12.740820576754516,
 -17.507785805392245,
 -21.046009647843405,
 -23.116848973689159,
 -23.581656657318561,
 -22.411230149139371,
 -19.687801579800933,
 -15.599433775114861,
 -10.427204083996045,
 -4.5260494203699295,
 1.6994221022151947,
 7.8225045461423832,
 13.423473455470585,
 18.118189385237798,
 21.584276371071994,
 23.583091319665698,
 23.975987142461676,
 22.733761326863444,
 19.938646040458238,
 15.77870414593751,
 10.535013029038282,
 4.5625096404499601,
 -1.7341978695731228,
 -7.9284035267658624,
 -13.600382838597284,
 -18.365996323573384,
 -21.902867980848058,
 -23.972354680698103,
 -24.43580929820844,
 -23.26402928448293,
 -20.539246770867223,
 -16.449524583869476,
 -11.275940073101438,
 -5.3734301531854101,
 0.85339706163294693,
 6.9778356330393905,
 12.580161104394932,
 17.276234030040044,
 20.743678444904262,
 22.743851254981934,
 23.13810537101719,
 21.897238279714841,
 19.10348214

In [148]:
def Fourier_model(x):
    col_names = [u'year', u'month', u'day', u'hour', u'temp', u'dew_temp', u'pressure',
       u'wind_angle', u'wind_speed', u'sky_code', u'rain_hour', u'rain_6hour', u'city']
    data = []
    nn = len(x)
    for i in range(nn):
        row = x[i]
        li = row.split()
        for d in range(12):
            li[d] = int(li[d])
        data.append(li)
    df = pd.DataFrame(data, columns=col_names)
    return fourier_model.predict(df) #fourier_model_pd.predict(df) 0.99...


grader.score('ts__fourier_model', Fourier_model)  #lambda x: [0] * len(x)

Your score:  1.00153001018


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*